# Importing the dataset into a dataframe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

import seaborn as sns

from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
import sklearn.model_selection as ms
import sklearn.preprocessing as preprocess
from sklearn.model_selection import train_test_split
import pickle
from sklearn.metrics import classification_report
    


In [ ]:
# !pip install pyarabic
# !pip install langdetect
# !pip install nltk

# from langdetect import detect
# import pyarabic.araby as araby
# nltk.download("all")


In [ ]:
dataset_folder_path="/home/youssef/AUC/Spring22/CSCE493002 - Machine Learning/project/repo/datasets"
df = pd.read_csv(dataset_folder_path+'/cleanedText.csv')
# df = pd.read_csv(dataset_folder_path+'/ar_reviews_100k.csv')
df.head()

In [ ]:
df_copy=df.copy()

In [ ]:
df_copy["text"]=df_copy['text'].values.astype('U')
df

# Preprocessing

In [ ]:
import numpy as np
from langdetect import detect
from nltk.stem.isri import ISRIStemmer
from nltk import word_tokenize
import pyarabic.araby as araby
# from sklearn.feature_extraction.text import TfidfVectorizer
import pickle


class Preprocessor:
    def __init__(self,vectorizer_path=None):
        self.vectorizer=None 
        if vectorizer_path:
            self.load_tfidf_vectorizer(vectorizer_path)
    
    def load_tfidf_vectorizer(self,path):
        file=open(path,'rb')
        self.vectorizer=pickle.load(file)
        file.close()
    
    def save_tfidf_vectorizer(self,path):
        if not self.vectorizer:
            return None
        file=open(path,"wb")
        pickle.dump(file,self.vectorizer)
        file.close()
    
    def preprocess(self,sentence,vectorizer_path=""):
#         if not self.vectorizer and vectorizer_path=="":
#             return None

        # if not self.detect_lang(sentence):
        #     print("sentence Not arabic!")
        #     return None
        
        result=self.stem(sentence)
        result=self.normalize(result)
        result=self.remove_redundant_words(result)



        return result


    def detect_lang(self,sentence):
        try:
            print("\n\nhere\n\n")
            language = detect(sentence)
            print("\n\nhere2\n\n")

            if language != 'ar':
                return False
        except:
                return False
        print("returning true")
        return True

    def stem(self,sentence):
        st = ISRIStemmer()
        stemmed_sentence=""
        for a in word_tokenize(sentence):
            stemmed=st.stem(a)
            stemmed_sentence+=(stemmed+" ")
        return stemmed_sentence
    
    def normalize(self,sentence):
        normalized=sentence
        normalized=araby.strip_tashkeel(normalized)
        normalized= araby.strip_tatweel(normalized)
        normalized=araby.normalize_hamza(normalized)
        return normalized
    
    

    def remove_redundant_words(self,sentence):
        stop_words=['من','على','عن','ب','ك','ل','فى','و','ان','هذا','او','كتب','...','.','','الى','فيه','انه','قبل','//','..','،',':',"؟",'/']
        words=sentence.split()
        resultWords= [word for word in words if word not in stop_words]
        return ' '.join(resultWords)
    
    def convert_to_tfidf(self,sentence):
        if not self.vectorizer:
            return None
        print(self.vectorizer)
        return self.vectorizer.transform([sentence])



# Shuffle

In [ ]:
#shuffling
from  sklearn.utils import shuffle
df_shuffled=shuffle(df_copy,random_state=0)
df_shuffled

In [ ]:


#Splitting data
x=df_shuffled['text']
y=np.expand_dims(df_shuffled['label'],axis=1)
X_train,X_test,Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=1)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=0.25,random_state=1)

print("Train: " ,X_train.shape,Y_train.shape,"\nValidation:",X_val.shape,Y_val.shape,"\nTest: ",(X_test.shape,Y_test.shape))

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)
tf_x_val=vectorizer.transform(X_val)

In [ ]:
vectorizer_file_path="tfidfVectorizerDump.joblib"
with open(vectorizer_file_path,"wb") as f:
        pickle.dump(vectorizer, f)

In [ ]:
print(tf_x_train.shape,tf_x_val.shape,tf_x_test.shape)

# Embedding

In [ ]:
# Python program to generate word vectors using Word2Vec
 
# importing all necessary modules
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import gensim
from gensim.models import Word2Vec

warnings.filterwarnings(action = 'ignore')
 

# iterate through each sentence in the file
def toknize_text(sentence):
    data = []
    for i in sent_tokenize(sentence):
        temp = []

        # tokenize the sentence into words
        for j in word_tokenize(i):
            temp.append(j.lower())

        data=temp
    return data


In [ ]:
df_shuffled['toknized']=df_shuffled['text'].apply(toknize_text)
df_shuffled

In [ ]:
toknized_text=df_shuffled['toknized']
preprocess=Preprocessor().preprocess
words=['انثى','ملك','ذكر']
words=[preprocess(word) for word in words]
pos=[words[1],words[0]]
negs=[words[2]]
print(negs)
words

In [ ]:
# Create CBOW model
model1 = gensim.models.Word2Vec(toknized_text, min_count = 1, vector_size = 300, window = 5)
model1.wv.most_similar(positive=pos,negative=negs,topn=10)     

In [ ]:
# Create Skip Gram model
model2 = gensim.models.Word2Vec(toknized_text, min_count = 1, vector_size = 300, window = 5, sg = 1,workers=20)
print(model2.wv.most_similar(positive=pos,negative=negs,topn=10))

In [ ]:
model2.wv.get_vector('ملك')

## Add Weight to the Embedding

In [ ]:
for index,row in df.iterrows():
    try:
        language = detect(row[1])
        if language != 'ar':
          row["text"]="-1"
    except:
        language = "error"
        print("Row ",index," does not contain Arabic words and will be removed")
        row["text"]="-1"
        #print(row["text"])

df = df[df['text']!="-1"] #remove non-arabic text
#df.info()

In [ ]:
def get_freq(sentence):
    if(type(sentence)==type(float(0.1))):
        print(sentence)
#   print(type(sentence))
#   freq=dict()
#   for word in sentence:
#       if word in freq:
#         freq[word]+=1
#       else:
#         freq[word]=1
#   return freq

In [ ]:
df['freq']=df['text'].apply(get_freq)
# df['freq']


In [ ]:
def parallelEmb1 (row):
    #print(row.name)
    freqDict=row["freq"]

    sentence = np.array(list(freqDict.keys()))
    
#     print(row.name)
    arr= [arrOfEmbeddings[ind]*freqDict[sentence[ind]]#tfidfDF[sentence[ind]][row.name] if sentence [ind] in tfidfDF.columns else 0#freqDict[sentence[ind]]
          for ind in range(len(sentence)) 
          if sentence[ind] in unique_dict.keys()]
    
    totalFreq = sum(freqDict.values())
    
    if len(arr) ==0:
       arr = list(np.zeros(300))
        
    arr=np.array(arr)
    
    vectorSum = np.sum(arr,axis=0)
    
    if (totalFreq == 0):
       vectorSum = list(np.zeros(300))
    else: 
       vectorSum = vectorSum * 1/totalFreq 
    return vectorSum

## Nerual Networks

In [ ]:
class AdamOptimizer:
    def __init__(self,beta1,beta2,alpha,eps=10e-8):
        self.beta1=beta1
        self.beta2=beta2
        self.alpha=alpha
        self.eps=eps
        self.ms=[]
        self.vs=[]
        
    def reset_params(self,layers):
        self.ms=[ 
                  [np.zeros_like(layer.W,dtype=np.float64), np.zeros_like(layer.b,dtype=np.float64)] 
                  for layer in layers      
                 ]
        self.vs=[ 
                  [np.zeros_like(layer.W,dtype=np.float64), np.zeros_like(layer.b,dtype=np.float64)] 
                  for layer in layers      
                 ]
        
    def update(self,layers,N):
        for i in range(len(layers)):
            self.ms[i][0]= self.beta1*self.ms[i][0]+(1.0-self.beta1)*layers[i].dW
            self.ms[i][1]= self.beta1*self.ms[i][1]+(1.0-self.beta1)*layers[i].db
            
            self.vs[i][0]= self.beta2*self.vs[i][0]+(1.0-self.beta2)*np.square(layers[i].dW)
            self.vs[i][1]= self.beta2*self.vs[i][1]+(1.0-self.beta2)*np.square(layers[i].db)

            denDW= np.sqrt((self.vs[i][0] + self.eps))
            denB=(np.sqrt((self.vs[i][1] + self.eps)))
            
            numDW=(-1 * self.alpha * self.ms[i][0])
            numB=(-1 * self.alpha * self.ms[i][1])
                    
            deltaW = np.array(numDW /denDW ,dtype=np.float64)
            deltab = np.array( numB/ denB  ,dtype=np.float64)
        
            layers[i].W +=  deltaW/N
            layers[i].b +=  deltab/N
        
class GradientDescent:
    def __init__(self,alpha):
        self.alpha=alpha
    def reset_params(self,layers):
        pass
    def update(self,layers,N):
        for i in range(len(layers)):
            # layers[i].dW=layers[i].dW/N
            # layers[i].db=layers[i].db/N
            layers[i].W = layers[i].W - self.alpha * (layers[i].dW/N)
            layers[i].b = layers[i].b - self.alpha * (layers[i].db/N)

In [ ]:
import numpy as np
import pickle


class Layer:
    
    ### activations
    def _relu(self,z):
        return np.maximum(0,z)
    def _diff_relu(self,z):
        dZ=np.array(z,copy=True)
        dZ[dZ<=0]=0
        dZ[dZ>0]=1
        return dZ
    
    def _identity(self,z):
        return z
    
    def _identity_diff(self,z):
        return np.ones_like(z)
    
    def _sigmoid(self,z):
        return (1/(1+np.exp(-1*z)))

    def _diff_sigmoid(self,z):
        return self._sigmoid(z)*(1-self._sigmoid(z))
    
    def _softmax(self,z):
        expZ= np.exp(z-np.max(z))
        return expZ/expZ.sum(axis=0, keepdims=True)
    def _diff_softmax(self,z):
        pass

    
    ###########

    def __init__(self,n_input,n_output, activation="identity",name=None):
        self.n_output= n_output
        self.n_input= n_input
        self.name= name
        
        if activation == "identity":
            self.activation = self._identity
            self.diff_act= self._identity_diff
        
        elif activation == "sigmoid":
            self.activation = self._sigmoid
            self.diff_act= self._diff_sigmoid
            
        elif activation == "softmax":
            self.activation=self._softmax
            self.diff_act=self._diff_softmax
        elif activation =="relu":
            self.activation=self._relu
            self.diff_act=self._diff_relu
            
        self.reset_params()
            
        
    def reset_params(self): 
        self.W= np.random.randn(self.n_output,self.n_input)*np.sqrt(2/self.n_input)
        self.b= np.random.randn(self.n_output,1)*np.sqrt(2/self.n_input)

        self.dW= np.zeros_like(self.W)
        self.db= np.zeros_like(self.b)
        
        self.Z= None
        self.Ai = None
    def print_shapes(self):
        print("W: ",self.W.shape)
        print("b: ",self.b.shape)
    
    def forward(self,Ai): #data dim 

        z =  np.add((self.W @ Ai),self.b)
        A = self.activation(z)

        
        self.Z = z
        self.Ai = Ai
        return A
    
    
    def backward(self,inp):
        
       
        act_diff = self.diff_act(self.Z)
        
        tmp = inp * act_diff
        
        bet = tmp @ self.Ai.T # vector of 1s
        
        
        e = np.ones((self.Ai.shape[1],1))
        db = tmp @ e

        self.dW = (self.dW + bet)
        self.db = self.db + db
        
        
        return self.W.T @ tmp
    
    def print_weights(self):
        print("\n###################")
        if(self.name):
            print("name: ",self.name)
        print("dW: ",self.dW, "W: ",self.W)
    
    def zeroing_delta(self):
        self.dW= np.zeros_like(self.W)
        self.db= np.zeros_like(self.b)


class NN:
    
    ########
    ## losses
    def _MSE(self,y,yhat):
        a=np.square(yhat-y)
        a=np.sum(a)
        b= 1/(2*y.shape[1])
        return a*b

    ## diff losses
    def _diff_MSE(self,y,yhat,X):
        return (yhat-y)
    
    def _binary_cross_entropy(self,y,yhat):
        arr= -(y*np.log(yhat)+(1-y)*np.log(1-yhat))
        return arr.mean()
        
    def _diff_binary_cross_entropy(self,y,yhat,X):
        dl_dyhat= -(y/(yhat) - (1-y)/(1-yhat))
        return dl_dyhat
 
    
    #########
    
    def __init__(self,optimizer=None,loss="binary_cross"):
        self.layers = []
        self.optimizer=optimizer
        self.loss_name=loss
        self.initialize_loss()
    
   
    def initialize_loss(self): 
        if(self.loss_name=="binary_cross"):
            self.loss=self._binary_cross_entropy
            self.loss_diff=self._diff_binary_cross_entropy
        elif self.loss_name=="MSE":
            self.loss=self._MSE
            self.loss_diff=self._diff_MSE
        
    
    def reset_layers(self):
            for layer in self.layers:
                layer.reset_params()
    
    def forward(self,x_train):
        a=x_train
        for layer in self.layers:
            a = layer.forward(a)
        return a
    
    def backward(self,input):
        gd = input
        for layer in self.layers[::-1]:
            gd = layer.backward(gd)
            
    def add_layer(self,n_input,n_output, activation="identity",name=None):
        self.layers.append(Layer(n_input,n_output, activation=activation,name=name))
    
    def batch(self,x,y,batch_size):
        x= x.copy()
        y=y.copy()
        reminder= x.shape[0] % batch_size


        for i in range(0,x.shape[0],batch_size):
            yield (x[i:i+batch_size],y[i:i+batch_size])
        
        if reminder !=0:
            yield (x[x.shape[0]-reminder:],y[x.shape[0]-reminder:] )
    
    def fit(self, x_train,y_train,validation_data=None,batch_size=32, epochs=5): #data dim is MxN .. M no of examples.. N no of dimension
        
        M = x_train.shape[0]

        no_of_batches= np.ceil(M/batch_size)
        if(validation_data):
            x_valid=validation_data[0]
            y_valid=validation_data[1]
        
        
        for i in range(epochs):
            
            print("Epoche {}/{}".format(i+1,epochs))
            self.optimizer.reset_params(self.layers)
            batches=self.batch(x_train,y_train,batch_size)
            losses=[]
            j=0
            for cur_x,cur_y in batches:
                
                cur_x=cur_x.T
                cur_y=cur_y.T
                
                y_hat= self.forward(cur_x)

                dl_dyhat = self.loss_diff(cur_y,y_hat,self.layers[-1].Ai)
                loss=self.loss(cur_y,y_hat)
                
                losses.append(loss)

                self.backward(dl_dyhat)
                
                if batch_size==1:
                    N= M
                else:
                    N=cur_x.shape[-1]
                
                self.optimizer.update(self.layers,N)

                # zeroing deltas
                for layer in self.layers:
                    layer.zeroing_delta()
                j+=1
                
            if validation_data:
                y_hat_val = self.forward(x_valid.T)
                loss_val= self.loss(y_valid.T,y_hat_val)
                print("val_loss: {}....".format(loss_val),end=" ")
                ######
                #calc metrics
            avg_loss= np.array(losses).mean()
            if(avg_loss<0.05):
                print("Stopping early because loss converged to a small number")
                print("losses avg=",avg_loss)
                break
            else: print("losses avg=",avg_loss)

                

        print("Finished....") 
            
            
        

    
    def predict(self,x_test): #data dim is NxD .. N no of examples.. D no of dimension
        y_hat= self.forward(x_test.T).T

        y_hat[y_hat>0.5]=1
        y_hat[y_hat<=0.5]=0
        return y_hat
                    
    def print_weights(self):
        for i in range(len(self.layers)):
            print("layer i= ",i,end=" ")
            self.layers[i].print_weights()
    def print_shapes(self):
        for layer in self.layers:
            layer.print_shapes()
    
    def save_model(self,path):
        model=[self.layers,self.optimizer,self.loss]

        file=open(path,"wb")
        print("dumped model: ",model)

        pickle.dump(model,file)

        file.close()

    def load_model(self,path):
        file=open(path,"rb")

        model=pickle.load(file)

        file.close()
        print("loaded model: ",model)
        
        self.layers,self.optimizer,self.loss=model
        self.initialize_loss()

    

In [ ]:
print(tf_x_train.shape)
print(tf_x_test.shape)
print(tf_x_val.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:

adam= AdamOptimizer(beta1 = 0.9,beta2 = 0.99,alpha=0.1,eps=0.001)
nn = NN(optimizer=adam)

nn.add_layer(tf_x_train.shape[1],64,activation="relu",name="l1")
nn.add_layer(64,32,activation = "relu",name="l2")
nn.add_layer(32,8,activation = "relu",name="l4")
nn.add_layer(8,1,activation = "sigmoid",name="l5")



In [ ]:
nn.fit(tf_x_train,Y_train,validation_data=[tf_x_val,Y_val],batch_size=32,epochs=1)
# nn.load_model("modelDump.joblib")


In [ ]:
y_pred=nn.predict(tf_x_test)

In [ ]:
print(len(y_pred))
np.unique(y_pred,return_counts=True)

In [ ]:
# Y_test
print(classification_report(y_pred,Y_test))


In [ ]:
#dump model
nn.save_model("modelDump.joblib")

In [ ]:
file=open("tfidfVectorizerDump.joblib","wb")
pickle.dump(file,vectorizer)